In [1]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
from requests_html import HTMLSession

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


In [2]:
url_list = []

In [3]:
# with open("output.txt", "w") as urls:
for i in range(1, 58):
    url = f'https://www.flipkart.com/search?q=laptop&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={i}'
    print(f"EXRACTING PAGE ############################ {i}")
    with rq.get(url=url) as cur_page:
        soup = BeautifulSoup(cur_page.text, "lxml")
        kk = None
        showing = None
        for div in soup.find_all("div"):
            for span in div.find_all("span"):
                if span.text.startswith("Showing"):
                    kk = span.next_element
                    showing = span.text
                    break
            if kk is not None:
                break
        if showing is not None:
            showing = showing.split()
            start = int(showing[1])
            end = int(showing[3])
            for a in kk.find_all_next("a"):
                # new_a = a
                # for div in new_a.find_all("div"):
                #     print(div.text)
                # break
                if start < end + 1:
                    start += 1
                    url_list.append("https://www.flipkart.com" + a["href"])
                    # urls.write("https://www.flipkart.com" + a["href"] + "\n")
                else:
                    print(f'EXTRACTED {start} URLs....')
                    break
        else:
            print("NO RESULTS FOUND, STOPPING...")
            break
print("STARTING DATA EXTRACTION FROM EACH URL")


EXRACTING PAGE ############################ 1
EXTRACTED 25 URLs....
EXRACTING PAGE ############################ 2
EXTRACTED 49 URLs....
EXRACTING PAGE ############################ 3
EXTRACTED 73 URLs....
EXRACTING PAGE ############################ 4
EXTRACTED 97 URLs....
EXRACTING PAGE ############################ 5
EXTRACTED 121 URLs....
EXRACTING PAGE ############################ 6
EXTRACTED 145 URLs....
EXRACTING PAGE ############################ 7
EXTRACTED 169 URLs....
EXRACTING PAGE ############################ 8
EXTRACTED 193 URLs....
EXRACTING PAGE ############################ 9
EXTRACTED 217 URLs....
EXRACTING PAGE ############################ 10
EXTRACTED 241 URLs....
EXRACTING PAGE ############################ 11
EXTRACTED 265 URLs....
EXRACTING PAGE ############################ 12
EXTRACTED 289 URLs....
EXRACTING PAGE ############################ 13
EXTRACTED 313 URLs....
EXRACTING PAGE ############################ 14
EXTRACTED 337 URLs....
EXRACTING PAGE ##################

In [4]:
def is_element_present(xpath: str, page: webdriver):
    try:
        WebDriverWait(page, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, xpath)))
    except TimeoutException:
        print("Element not present on page")
        raise TimeoutException
    return True


In [5]:
specs_list = []
price_list = []

In [6]:
options = Options()
# options.headless = True
options.add_argument('--headless=new')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [7]:
from multiprocessing import cpu_count

c_count = cpu_count()*2

step = len(url_list)//c_count
start = 0
end = step
url_list_split = []

for _ in range(c_count):
    url_list_split.append(url_list[start:end])
    start += step
    end += step

if len(url_list) % c_count != 0:
    k = 0
    for i in range(step*c_count, len(url_list)):
        url_list_split[k].append(url_list[i])
        k += 1


In [8]:
def scrap_urls(driver, cur_url_chunk):
    sp_list = []
    for i, url in enumerate(cur_url_chunk):
        found_spec = False
        # print(url)
        driver.get(url)
        cur_spec = []
        # is_element_present("_25b18c", driver)
        # print(f'FOR {i}')
        cur_page = driver.page_source
        soup = BeautifulSoup(cur_page, "lxml")
        # price = ""
        div = soup.select("div._1UhVsV")
        for tr in div[0].find_all("td"):
            found_spec = True
            # print(tr.text)
            cur_spec.append(tr.text)
            # pass

        if not found_spec:
            print(f"SPEC NOT FOUND FOR {i} , url :: {url}")
        cur_spec.append("price")
        price = soup.select("div._30jeq3._16Jk6d")
        # print(f'for {i}', price[0].text)
        cur_spec.append(price[0].text.replace(",", "").replace("₹", ""))
        sp_list.append(cur_spec)
        # driver.close()
    return sp_list


In [9]:
import asyncio
from concurrent.futures import ThreadPoolExecutor


async def async_operation(url_list_split, driver, num_executor):
    try:
        with ThreadPoolExecutor(max_workers=num_executor) as executor:
            loop = asyncio.get_event_loop()
            specs = [
                loop.run_in_executor(executor, scrap_urls,
                                     *(driver, cur_url_chunk))
                for cur_url_chunk in url_list_split
            ]
            # specs = await asyncio.gather(*specs)
            return specs
    except Exception as err:
        print(err)


In [10]:
import nest_asyncio
nest_asyncio.apply()


In [11]:
import asyncio


loop = asyncio.get_event_loop()

specs_list = loop.run_until_complete(async_operation(
    url_list_split, driver, num_executor=c_count))

# loop.close()


In [17]:
specs_list_final = await asyncio.gather(*specs_list)


In [18]:
data_dict = {}
col_dict = {}
col_set = set()

count = 0
for i, cur_spec_l in enumerate(specs_list_final):
    for j, cur_spec in enumerate(cur_spec_l):
        # print(cur_spec)
        col_list = []
        for index in range(0, len(cur_spec), 2):
            col_list.append(cur_spec[index])
            col_set.add(cur_spec[index])
            if cur_spec[index] not in data_dict.keys():
                data_dict[cur_spec[index]] = []
        col_dict[count] = col_list
        count += 1


In [27]:
import numpy as np
count = 0
for i, cur_spec_l in enumerate(specs_list_final):
    for j, cur_spec in enumerate(cur_spec_l):
        cur_col_list = col_dict[count]
        count += 1
        remaining_col_for_cur_index = col_set.symmetric_difference(
            set(cur_col_list))
        for index in range(0, len(cur_spec), 2):
            # print(index, cur_spec[index], i)
            data_dict[cur_spec[index]].append(cur_spec[index + 1])
        for col in remaining_col_for_cur_index:
            data_dict[col].append(np.nan)


In [28]:
dataframe = pd.DataFrame(data_dict)
dataframe.to_csv("flipkart_laptop_data.csv")
print("EXTRACTION COMPLETED...")
print(dataframe.isna().sum())


EXTRACTION COMPLETED...
Sales Package         0
Model Number          0
Part Number           0
Model Name           50
Series              126
                   ... 
Wireless WAN        959
Recovery Options    975
Brightness          974
Stylus Included     976
Inbuilt 4G LTE      977
Length: 91, dtype: int64
